# CS336 Assignments

| # | Topic                         | Description                                 |
|---|-------------------------------|---------------------------------------------|
| 1 | Basics                        | Train an LLM from scratch                   |
| 2 | Systems                       | Make it run fast!                           |
| 3 | Scaling                       | Make it performant at a FLOP budget         |
| 4 | Data                          | Prepare the right datasets                  |
| 5 | Alignment & Reasoning RL      | Align it to real-world use cases            |

### Assignment #1
- Implement all of the components (tokenizer, model, loss function, optimizer) necessary to train a standard Transformer language model
- Train a minimal language model

In [1]:
import warnings
warnings.filterwarnings("ignore")

import torch
import lovely_tensors as lt
lt.monkey_patch()

import tiktoken

from datasets import load_dataset
import joblib

from torch.utils.data import TensorDataset, DataLoader


In [5]:
from string import ascii_letters

{l: ord(l) for l in ascii_letters}

{'a': 97,
 'b': 98,
 'c': 99,
 'd': 100,
 'e': 101,
 'f': 102,
 'g': 103,
 'h': 104,
 'i': 105,
 'j': 106,
 'k': 107,
 'l': 108,
 'm': 109,
 'n': 110,
 'o': 111,
 'p': 112,
 'q': 113,
 'r': 114,
 's': 115,
 't': 116,
 'u': 117,
 'v': 118,
 'w': 119,
 'x': 120,
 'y': 121,
 'z': 122,
 'A': 65,
 'B': 66,
 'C': 67,
 'D': 68,
 'E': 69,
 'F': 70,
 'G': 71,
 'H': 72,
 'I': 73,
 'J': 74,
 'K': 75,
 'L': 76,
 'M': 77,
 'N': 78,
 'O': 79,
 'P': 80,
 'Q': 81,
 'R': 82,
 'S': 83,
 'T': 84,
 'U': 85,
 'V': 86,
 'W': 87,
 'X': 88,
 'Y': 89,
 'Z': 90}

In [6]:
chr(115)

's'

### Exercise 1: Problem (unicode1): Understanding Unicode (1 point)


In [11]:
chr(0)

'\x00'

This represents a null character often used to represent end of a string. It is also called an escape sequence.

In [22]:
repr('\x00')

"'\\x00'"

The string representation of this character is '\x00'. When this string is passed to print function, it's rendered as null as that is the purpose of this character.

In [13]:
chr(0)

'\x00'

In [14]:
print(chr(0))

 


In [15]:
"this is a test" + chr(0) + "string"

'this is a test\x00string'

In [16]:
print("this is a test" + chr(0) + "string")

this is a test string


When we print the character with the print function, the character is executed and hence renders nothing on the stdout.

### Exercise 2: Problem (unicode2): Unicode Encodings (3 points)

In [32]:
test_string = "Hello"
test_string_encoded = test_string.encode("UTF-8")
print(f"test_string_encoded: {test_string_encoded}")
print(f"Byte values: {list(test_string_encoded)}")
print(f"test string decoded: {test_string_encoded.decode("UTF-8")}")

test_string_encoded: b'Hello'
Byte values: [72, 101, 108, 108, 111]
test string decoded: Hello


(a) What are some reasons to prefer training our tokenizer on UTF-8 encoded bytes, rather than
UTF-16 or UTF-32? It may be helpful to compare the output of these encodings for various
input strings

A: Majority of the internet comprises of UTF-8 characters. And, UTF-8 is space efficient as 5 characters in UTF-8 takes 5 bytes whereas UTF-16 and UTF-32 takes 2x and 4x the bytes.

In [ ]:
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):
    return "".join([bytes([b]).decode("utf-8") for b in bytestring])

decode_utf8_bytes_to_str_wrong("hello".encode("utf-8"))

'hello'

In [49]:
decode_utf8_bytes_to_str_wrong("café".encode("utf-8"))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc3 in position 0: unexpected end of data

The function attempts to convert each character as a standalone single character. 

In [51]:
"café".encode("UTF-8")

b'caf\xc3\xa9'